### Initialize library

In [1]:
%load_ext autoreload
%autoreload 2

from olp_library import *
# import FOON_graph_analyser as fga

### Language to  OLP

In [2]:
models = [
    "gpt-3.5-turbo",
    "gpt-4",
    ]

in_context_file = "incontext_examples.txt"
user_task = "How do I make coffee?"
# user_task = "How do I make a burger?"
# user_task = "How do I unlock a door?"
# user_task = "How do I turn on the lights in my house"

high_level_plan, object_level_plan, plan_objects = generate_olp(user_task, in_context_file, models[1], verbose=False)

In [3]:
## Visualize Outputs
print("***********************\n   High level plan\n***********************\n",   high_level_plan,   "\n")
print("***********************\n   Object level plan\n***********************")
print(*object_level_plan, sep="\n")
print("\n***********************\n   Plan objects\n***********************\n",      plan_objects,      "\n")

***********************
   High level plan
***********************
 1. Fill kettle with water.
2. Heat kettle until water boils.
3. Place coffee grounds in coffee filter.
4. Put coffee filter in coffee maker.
5. Pour boiled water into coffee maker.
6. Turn on coffee maker to brew coffee.
7. Pour brewed coffee into cup.

unique_objects: kettle, water, coffee grounds, coffee filter, coffee maker, cup. 

***********************
   Object level plan
***********************
{'Step': 'Fill kettle with water', 'Objects': ['kettle', 'water'], 'Action': 'Fill', 'StateChanges': {'kettle': {'Precondition': ['empty'], 'Effect': ['contains water']}, 'water': {'Precondition': ['not in kettle'], 'Effect': ['in kettle']}}}
{'Step': 'Heat kettle until water boils', 'Objects': ['kettle', 'water'], 'Action': 'Heat', 'StateChanges': {'kettle': {'Precondition': ['contains water', 'not hot'], 'Effect': ['contains water', 'hot']}, 'water': {'Precondition': ['in kettle', 'not boiling'], 'Effect': ['in kettle'

In [11]:
## Test Foon creation
plan_step = object_level_plan[0]

sample_unit = create_olp_functionalUnit(plan_step)
sample_unit.print_functions[2](version=1)

Creating functional unit for: Fill kettle with water
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['empty'], 'Effect': ['contains water']}
		 Precondition : ['empty'] || related objects: []
		 Effect : ['contains water'] || related objects: ['water']
Current object is : water
	 water state changes: {'Precondition': ['not in kettle'], 'Effect': ['in kettle']}
		 Precondition : ['not in kettle'] || related objects: ['kettle']
		 Effect : ['in kettle'] || related objects: ['kettle']
O	kettle	None
S	contains water	{water}
O	water	None
S	not in kettle	 [kettle]
S	in kettle	 [kettle]
M	Fill	<Assumed>
O	kettle	None
S	contains water	{water}
O	water	None
S	not in kettle	 [kettle]
S	in kettle	 [kettle]


In [10]:
#Complete FOON prototype:
FOON_prototype = []

for step in object_level_plan:
    # -- now we will create functional units that follow the FOON format:
    new_unit = create_olp_functionalUnit(step)      
    new_unit.print_functions[2](version=1)
    
    # input()
    print()

    # -- add the functional unit to the FOON prototype:
    FOON_prototype.append(new_unit)

Creating functional unit for: Fill kettle with water
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['empty'], 'Effect': ['contains water']}
		 Precondition : ['empty'] || related objects: []
		 Effect : ['contains water'] || related objects: ['water']
Current object is : water
	 water state changes: {'Precondition': ['not in kettle'], 'Effect': ['in kettle']}
		 Precondition : ['not in kettle'] || related objects: ['kettle']
		 Effect : ['in kettle'] || related objects: ['kettle']
O	kettle	None
S	contains water	{water}
O	water	None
S	not in kettle	 [kettle]
S	in kettle	 [kettle]
M	Fill	<Assumed>
O	kettle	None
S	contains water	{water}
O	water	None
S	not in kettle	 [kettle]
S	in kettle	 [kettle]

Creating functional unit for: Heat kettle until water boils
Objects: ['kettle', 'water']
Current object is : kettle
	 kettle state changes: {'Precondition': ['contains water', 'not hot'], 'Effect': ['contains water', 'hot']}
		 Precondition : ['